### 什么是 RLHF（Reinforcement Learning from Human Feedback）

RLHF 是用“人类偏好”来对大语言模型进行对齐的一套训练范式：先让模型会做事，再让模型知道“什么更好”，最后用强化学习把“更好”的偏好真正优化进生成策略里。

- **目标**：让模型更符合人类意图、更安全、更有用
- **核心思想**：
  - 用监督微调（SFT）教会模型基本的指令跟随
  - 用偏好数据训练奖励模型（RM），学会打分“更好/更差”的回答
  - 用强化学习（PPO）在奖励信号下优化策略，权衡质量、稳定性与多样性
- **关键组件**：指令数据、偏好数据（A/B 对比）、奖励模型、强化学习算法、KL 约束/参考策略
- **典型产物**：
  - SFT 模型（会做事）
  - RM 奖励模型（会打分）
  - PPO 后的对齐模型（做得更好）
  - DPO （取缔RM+PPO）


### 三、RLHF 的三阶段流程（工程化视角）

| 阶段 | 名称 | 作用 | 技术 |
|---|---|---|---|
| 1️⃣ | SFT（监督微调） | 教模型执行指令 | CrossEntropyLoss |
| 2️⃣ | Reward Model 训练 | 学会“什么样的回答更好” | Pairwise ranking (A > B) |
| 3️⃣ | PPO 强化优化 | 用奖励信号优化生成策略 | PPO 算法（Policy Gradient） |

#### 1️⃣ SFT（监督微调）
- **输入**：指令-回答对（高质量、人类书写/筛选）
- **目标**：让模型基本学会“按指令作答”
- **训练**：最小化交叉熵损失（参考常用指令数据集）
- **输出**：SFT 模型（作为后续 RM/PPO 的参考策略）

#### 2️⃣ 奖励模型（RM）训练
- **输入**：同一指令下成对回答（A、B），以及偏好标签（A > B）
- **目标**：学习“偏好评分函数” r(x, y)
- **训练**：Pairwise ranking（如 Bradley–Terry/Logistic loss）
- **输出**：能对任意回答打分的奖励模型

#### 3️⃣ PPO 强化优化
- **输入**：SFT 模型作为初始策略 π_θ，奖励模型 r 作为奖励信号
- **目标**：在 KL 约束下最大化期望奖励，提升对齐度与有用性
- **训练**：PPO（剪切策略梯度），引入 KL 惩罚以保持与参考策略接近
- **输出**：PPO 后的对齐模型（更符合人类偏好）

> 实践要点：高质量偏好数据与稳定的 KL 控制是成功关键；监控长度偏置、模式坍缩与过拟合。

#### DPO（Direct Preference Optimization）
- **定位**：作为第 3 阶段（PPO）的常见替代方案，用偏好对直接优化策略。
- **核心**：基于 `(x, y_pos, y_neg)` 提高 `y_pos` 概率、降低 `y_neg`，并以参考策略 `π_ref` 的对数概率差作隐式 KL 约束。
- **直观目标**：最小化 `-log σ(β[(log πθ(y_pos|x) - log πθ(y_neg|x)) - (log πref(y_pos|x) - log πref(y_neg|x))])`
- **优点**：流程简单、无奖励模型与 RL 回路、稳定易复现、吞吐高。
- **局限**：依赖高质量偏好数据；极端分布迁移下可控性较弱。


### 实验设置：模型与数据集选择

- 模型：`Qwen2.5-1.5B-Instruct`（中文指令能力强，小参数、易于 LoRA/QLoRA）
- SFT 数据：`BelleGroup/train_0.5M_CN`（中文指令-回答对，体量适中，可采样）
- 偏好数据（用于 DPO/RM）：`argilla/ultrafeedback-binarized-preferences`（成对偏好，易直接用于 DPO）

下面先安装依赖并加载模型、抽样加载 SFT 数据（少量样本用于快速跑通）。


In [1]:
# 安装依赖（仅需首次）
%pip -q install transformers>=4.44.0 accelerate datasets peft bitsandbytes trl>=0.9.6 sentencepiece



zsh:1: 4.44.0 not found
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "Qwen/Qwen2.5-1.5B-Instruct"

use_cuda = torch.cuda.is_available()
use_mps = torch.backends.mps.is_available()

quant_config = None
try:
    if use_cuda:
        from transformers import BitsAndBytesConfig  # 仅在 CUDA 下尝试 4bit
        import importlib.metadata as im
        im.version("bitsandbytes")  # 检查安装
        quant_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
        print("[Info] Using bitsandbytes 4-bit on CUDA.")
    else:
        print("[Info] CUDA 不可用，跳过 bitsandbytes 量化，改用 MPS/CPU.")
except Exception as e:
    print(f"[Warn] bitsandbytes 不可用或未安装：{e}. 将使用非量化加载。")

# 设备映射
if use_cuda:
    device_map = "auto"
    dtype = torch.bfloat16
elif use_mps:
    device_map = {"": "mps"}
    dtype = torch.float16
else:
    device_map = {"": "cpu"}
    dtype = torch.float32

# 加载 tokenizer / model（按可用性量化）
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, trust_remote_code=True)

load_kwargs = dict(
    device_map=device_map,
    torch_dtype=dtype,
    trust_remote_code=True,
)
if quant_config is not None:
    load_kwargs["quantization_config"] = quant_config

model = AutoModelForCausalLM.from_pretrained(model_name, **load_kwargs)

print(f"[Device] cuda={use_cuda}, mps={use_mps}, dtype={dtype}")

# 快速自检
inputs = tokenizer("你好，简要介绍一下你自己。", return_tensors="pt")
if use_mps:
    inputs = {k: v.to("mps") for k, v in inputs.items()}
else:
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

with torch.inference_mode():
    out = model.generate(**inputs, max_new_tokens=64, do_sample=False)
print(tokenizer.decode(out[0], skip_special_tokens=True))



[Info] CUDA 不可用，跳过 bitsandbytes 量化，改用 MPS/CPU.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


[Device] cuda=False, mps=True, dtype=torch.float16
你好，简要介绍一下你自己。 作为一个AI助手，我叫通义千问。我可以回答各种问题、提供信息和帮助您完成任务。有什么我能帮您的吗？


In [1]:
from datasets import load_dataset

def _to_sft(example):
    instr = example.get("instruction", "")
    inp = example.get("input", "")
    output = example.get("output", None)
    prompt = (instr + ("\n" + inp if inp else "")).strip()
    return {"prompt": prompt, "response": output}

# SFT：抽样加载 BELLE 中文指令数据
sft_ds = load_dataset("BelleGroup/train_0.5M_CN", split="train[:2000]")
sft_ds = sft_ds.map(_to_sft, remove_columns=sft_ds.column_names)
print("SFT 条数:", len(sft_ds))
print("SFT 样本示例:", sft_ds[0])

# 偏好数据：UltraFeedback（用于 DPO/RM）优先使用官方版本

def load_ultrafeedback(max_rows="20000"):
    try:
        ds = load_dataset("HuggingFaceH4/ultrafeedback_binarized", split=f"train_prefs[:{max_rows}]")
    except Exception:
        ds = load_dataset("argilla/ultrafeedback-binarized-preferences", split=f"train[:{max_rows}]")
    return ds

pref = load_ultrafeedback()

# 统一映射到 (prompt, y_pos, y_neg)

def _to_pref(ex):
    prompt = ex.get("prompt") or ex.get("question") or ex.get("instruction") or ex.get("input")
    y_pos = ex.get("chosen") or ex.get("better_response") or ex.get("pos") or ex.get("preferred")
    y_neg = ex.get("rejected") or ex.get("worse_response") or ex.get("neg") or ex.get("other")
    return {"prompt": prompt, "y_pos": y_pos, "y_neg": y_neg}

pref = pref.map(_to_pref)
pref = pref.filter(lambda e: isinstance(e["prompt"], str) and isinstance(e["y_pos"], str) and isinstance(e["y_neg"], str))

print("偏好条数:", len(pref))
if len(pref) > 0:
    sample = pref[0]
    preview = {k: (sample[k][:120] + "...") for k in ["prompt", "y_pos", "y_neg"]}
    print("偏好样本示例:", preview)
else:
    print("[Warn] 偏好数据为空，请增大抽样量或检查数据集网络可用性。可尝试将 max_rows 提升到 '50000'。")



SFT 条数: 2000
SFT 样本示例: {'prompt': '给定一个英文句子，翻译成中文。\nI love to learn new things every day.', 'response': '我每天喜欢学习新事物。'}


data/train_prefs-00000-of-00001.parquet:   0%|          | 0.00/226M [00:00<?, ?B/s]

Cancellation requested; stopping current tasks.


KeyboardInterrupt: 